In [9]:
import numpy as np
import pandas as pd
import optuna
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

# Supress Optuna's huge log output to keep your console clean
optuna.logging.set_verbosity(optuna.logging.WARNING)


In [2]:
def make_lags(df, cols, lags):
    return pd.concat([df[col].shift(lag).rename(f"{col}_L{lag}") for col in cols for lag in lags], axis=1)

In [3]:
df = pd.read_csv("../../../data/cleaned/multi_var_wind_hourly_cleaned.csv")
df = df.sort_values('datetime').reset_index(drop=True)
df['wind_speed'] = df['wind_speed'].astype(float)

In [4]:
df.head()

,datetime,pressure,temperature,humidity,wind_direction,wind_speed,u,v,wind_speed_reconstructed,pressure_24h
0,2024-01-01 00:00:00,758.466667,-1.016667,62.166667,137.000000,0.100000,-0.022693,-0.047340,0.052498,758.466667
1,2024-01-01 01:00:00,758.383333,-1.533333,64.000000,207.166667,0.283333,-0.113825,0.011167,0.114371,758.425000
2,2024-01-01 02:00:00,758.383333,-1.150000,61.333333,63.000000,0.200000,-0.169286,-0.061781,0.180207,758.411111
3,2024-01-01 03:00:00,758.783333,-1.166667,58.666667,181.166667,0.500000,0.142045,-0.397690,0.422296,758.504167
4,2024-01-01 04:00:00,759.000000,-1.483333,62.166667,109.666667,0.966667,-0.512590,-0.609574,0.796448,758.603333


In [5]:
df = df[['datetime', 'pressure', 'temperature', 'humidity', 'wind_speed', 'u', 'v']]

In [7]:
# ensure the column is datetime *type*
df['datetime'] = pd.to_datetime(df['datetime'])   # or df['datetime'] if that's the name

# now extract hour
df['hour'] = df['datetime'].dt.hour
df['sin_h'] = np.sin(2 * np.pi * df['hour'] / 24)
df['cos_h'] = np.cos(2 * np.pi * df['hour'] / 24)

In [10]:
# --- 1. Define the Objective Function ---
def objective(trial, X, y):
    """
    Optuna will run this function multiple times with different hyperparams 
    to minimize the return value (MAE).
    """
    
    # Define the search space (Bayesian distributions)
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'max_depth': trial.suggest_int('max_depth', 3, 8), # Keep low for small data
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        # Fixed params
        'objective': 'reg:absoluteerror',
        'n_jobs': -1,
        'random_state': 42
    }

    # Inner Cross-Validation (TimeSeriesSplit)
    # We use this to ensure the params don't overfit
    tscv = TimeSeriesSplit(n_splits=3)
    scores = []

    for train_index, val_index in tscv.split(X):
        X_train_cv, X_val_cv = X[train_index], X[val_index]
        y_train_cv, y_val_cv = y[train_index], y[val_index]

        model = xgb.XGBRegressor(**params)
        model.fit(X_train_cv, y_train_cv)
        
        preds = model.predict(X_val_cv)
        mae = mean_absolute_error(y_val_cv, preds)
        scores.append(mae)

    # Return the average MAE of the folds
    return np.mean(scores)


In [13]:
# --- 2. Integration into Your Rolling Loop ---

# 1. REVISED LAGS (Option 2)
# Keep recent history (1,2,3) and the exact time yesterday (24). 
# Drop the noisy lags (23, 25, 48).
LAGS = [1, 2, 3, 24] 
cols_to_lag = ['pressure', 'temperature', 'wind_speed', 'u', 'v']

# 2. Apply Lags
df_lags = make_lags(df, cols_to_lag, LAGS)
df_final = pd.concat([df, df_lags], axis=1)

# 3. Add Rolling Features (Trend & Volatility)
# Calculated on the CURRENT available data (at time t)
# This captures the "average weather of the last day" and "gustiness"
df_final['wind_rolling_24_mean'] = df_final['wind_speed'].rolling(window=24).mean()
df_final['wind_rolling_24_std'] = df_final['wind_speed'].rolling(window=24).std()

# 4. Target Creation & Cleanup
df_final['target_wind_speed'] = df_final['wind_speed'].shift(-1)
df_final.dropna(inplace=True)
X = df_final.drop(['datetime', 'target_wind_speed'], axis=1)
y = df_final['target_wind_speed']


# 3. train/test split (last 720 rows locked)
train_end = len(df_final) - 720
test_df = df_final.iloc[train_end:].copy()

current_obs = ['pressure', 'temperature', 'u', 'v', 'wind_speed']
FEATS = [c for c in df_final.columns if '_L' in c] + ['sin_h', 'cos_h'] + current_obs



In [15]:
RETRAIN_INTERVAL = 168 # hours (weekly)
TUNING_WINDOW_HOURS = 2160 # Approx 3 months for tuning
N_ITER_RANDOM_SEARCH = 15 # Number of param combinations to try (faster)
N_SPLITS_INNER_CV = 3 # Fewer splits for speed during tuning


In [16]:
print("Starting rolling forecast with Bayesian Optimization...")

preds = []
model = None
current_best_params = {}

# Number of Bayesian trials to run per re-tuning (Faster than Random Search)
N_TRIALS = 20 

for t in range(train_end, len(df_final)):

    # Check if it's time to retrain and re-tune
    if (t - train_end) % RETRAIN_INTERVAL == 0:
        print(f"\n--- [Step {t}] Retraining & Bayesian Tuning ---")
        
        # --- A. RE-TUNING (Bayesian Optimization) ---
        tune_start_idx = max(0, t - TUNING_WINDOW_HOURS)
        tune_df = df_final.iloc[tune_start_idx:t]
        
        X_tune = tune_df[FEATS].values
        y_tune = tune_df['target_wind_speed'].values
        
        # Create the Study
        # direction='minimize' because we want to lower MAE
        study = optuna.create_study(direction='minimize')
        
        # Run optimization
        # We use a lambda function to pass X_tune and y_tune into the objective
        study.optimize(lambda trial: objective(trial, X_tune, y_tune), n_trials=N_TRIALS)
        
        current_best_params = study.best_params
        
        # Add fixed params back in for the final model
        current_best_params['objective'] = 'reg:absoluteerror'
        current_best_params['n_jobs'] = -1
        current_best_params['random_state'] = 42
        
        print(f"Best MAE found: {study.best_value:.4f}")
        print(f"Params: {study.best_params}")

        # --- B. RE-TRAINING (Sliding Window) ---
        # Note: Ensure you updated this to use your SLIDING logic from previous steps
        train_start_idx = max(0, t - 4300) # Example 6-month slide
        train_window_df = df_final.iloc[train_start_idx:t]
        
        X_train = train_window_df[FEATS].values
        y_train = train_window_df['target_wind_speed'].values
        
        model = xgb.XGBRegressor(**current_best_params)
        model.fit(X_train, y_train)

    # --- C. PREDICTION ---
    if model is None:
        preds.append(np.nan)
        continue

    x_next = df_final.iloc[[t]][FEATS].values
    preds.append(model.predict(x_next)[0])



Starting rolling forecast with Bayesian Optimization...

--- [Step 9584] Retraining & Bayesian Tuning ---
Best MAE found: 0.8057
Params: {'n_estimators': 790, 'max_depth': 3, 'learning_rate': 0.011909773348693194, 'subsample': 0.619920898956543, 'colsample_bytree': 0.9948378752264485, 'gamma': 3.390855952997832, 'reg_alpha': 0.09756396695463336, 'reg_lambda': 0.48610569956155697}

--- [Step 9752] Retraining & Bayesian Tuning ---
Best MAE found: 0.8436
Params: {'n_estimators': 643, 'max_depth': 3, 'learning_rate': 0.03594641214399805, 'subsample': 0.8889972038059877, 'colsample_bytree': 0.6600415008414701, 'gamma': 3.4876485932460763, 'reg_alpha': 0.13959236199644096, 'reg_lambda': 0.19873400051203285}

--- [Step 9920] Retraining & Bayesian Tuning ---
Best MAE found: 0.6862
Params: {'n_estimators': 423, 'max_depth': 3, 'learning_rate': 0.023118999795277893, 'subsample': 0.7218841747940781, 'colsample_bytree': 0.9306567240750312, 'gamma': 3.8749503721711287, 'reg_alpha': 2.13383651626025

In [17]:
import sklearn.metrics as m

In [18]:

# --- 4. Evaluation ---
test_df['pred_xgb_dynamic'] = preds
# Drop any initial NaNs if we had them
test_df = test_df.dropna(subset=['pred_xgb_dynamic'])

print("\n--- Final Results ---")
print("XGB MAE with Dynamic Re-Tuning:", 
      m.mean_absolute_error(test_df['target_wind_speed'], test_df['pred_xgb_dynamic']))


--- Final Results ---
XGB MAE with Dynamic Re-Tuning: 0.6476247232021005
